In [1]:
import pandas as pd
import numpy as np
import scipy
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn import ensemble
from sklearn import datasets
from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error

/home/diegofvargas/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
df = pd.read_csv((
    "https://raw.githubusercontent.com/Thinkful-Ed/data-201-resources/"
    "master/ESS_practice_data/ESSdata_Thinkful.csv")).dropna()

In [7]:
df.head(20)

,cntry,idno,year,tvtot,ppltrst,pplfair,pplhlp,happy,sclmeet,sclact,gndr,agea,partner
0,CH,5.0,6,3.0,3.0,10.0,5.0,8.0,5.0,4.0,2.0,60.0,1.0
1,CH,25.0,6,6.0,5.0,7.0,5.0,9.0,3.0,2.0,2.0,59.0,1.0
2,CH,26.0,6,1.0,8.0,8.0,8.0,7.0,6.0,3.0,1.0,24.0,2.0
3,CH,28.0,6,4.0,6.0,6.0,7.0,10.0,6.0,2.0,2.0,64.0,1.0
4,CH,29.0,6,5.0,6.0,7.0,5.0,8.0,7.0,2.0,2.0,55.0,1.0
6,CH,40.0,6,3.0,0.0,5.0,2.0,0.0,2.0,2.0,1.0,76.0,1.0
7,CH,41.0,6,2.0,4.0,5.0,3.0,10.0,5.0,2.0,2.0,30.0,1.0
8,CH,51.0,6,2.0,8.0,8.0,8.0,9.0,6.0,4.0,2.0,84.0,2.0
9,CH,53.0,6,4.0,4.0,4.0,8.0,7.0,4.0,2.0,2.0,62.0,1.0
10,CH,55.0,6,1.0,6.0,7.0,7.0,9.0,5.0,2.0,2.0,33.0,1.0


In [9]:
df= df.drop('idno',1)

In [18]:
# Definine outcome and predictors.
# Set our outcome to 0 and 1.
y = df['partner'] - 1
X = df.loc[:, ~df.columns.isin(['partner', 'cntry', 'idno'])]

# Make the categorical variable 'country' into dummies.
X = pd.concat([X, pd.get_dummies(df['cntry'])], axis=1)

# Create training and test sets.
offset = int(X.shape[0] * 0.7)

# Put 70% of the data in the training set.
X_train, y_train = X[:offset], y[:offset]

# And put 30% in the test set.
X_test, y_test = X[offset:], y[offset:]

In [39]:
# We'll make 500 iterations, use 2-deep trees, and set our loss function.
params = {'n_estimators': 10000,
          'max_depth': 3,
          'loss': 'deviance'}

# Initialize and fit the model.
clf = ensemble.GradientBoostingClassifier(**params)
clf.fit(X_train, y_train)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=10000,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False)

In [36]:
predict_train = clf.predict(X_train)
predict_test = clf.predict(X_test)

In [37]:
# Accuracy tables.
table_train = pd.crosstab(y_train, predict_train, margins=True)
table_test = pd.crosstab(y_test, predict_test, margins=True)

train_tI_errors = table_train.loc[0.0,1.0] / table_train.loc['All','All']
train_tII_errors = table_train.loc[1.0,0.0] / table_train.loc['All','All']

test_tI_errors = table_test.loc[0.0,1.0]/table_test.loc['All','All']
test_tII_errors = table_test.loc[1.0,0.0]/table_test.loc['All','All']

print((
    'Training set accuracy:\n'
    'Percent Type I errors: {}\n'
    'Percent Type II errors: {}\n\n'
    'Test set accuracy:\n'
    'Percent Type I errors: {}\n'
    'Percent Type II errors: {}'
).format(train_tI_errors, train_tII_errors, test_tI_errors, test_tII_errors))

Training set accuracy:
Percent Type I errors: 0.03735531392493862
Percent Type II errors: 0.13048053314626448

Test set accuracy:
Percent Type I errors: 0.09488752556237219
Percent Type II errors: 0.18650306748466258


In [22]:
#Original Results
#Training set accuracy:
#Percent Type I errors: 0.04507190459487899
#Percent Type II errors: 0.17713083128726761

#Test set accuracy:
#Percent Type I errors: 0.05521472392638037
#Percent Type II errors: 0.19631901840490798


In [26]:
#N_estimators = 10000
#Training set accuracy:
#Percent Type I errors: 0.03139249386180288
#Percent Type II errors: 0.1241669589617678

#Test set accuracy:
#Percent Type I errors: 0.09161554192229039
#Percent Type II errors: 0.18732106339468302

In [31]:
#N_estimators = 10000 and Tree depth = 4
#Training set accuracy:
#Percent Type I errors: 0.0
#Percent Type II errors: 0.0

#Test set accuracy:
#Percent Type I errors: 0.14314928425357873
#Percent Type II errors: 0.16319018404907976
#Did worse on the test set, seems like there is overfitting going on due to the decision trees

In [38]:
#N_estimators = 10000 and loss = exponential
#Training set accuracy:
#Percent Type I errors: 0.03735531392493862
#Percent Type II errors: 0.13048053314626448

#Test set accuracy:
#Percent Type I errors: 0.09488752556237219
#Percent Type II errors: 0.18650306748466258
#slightly worse performance with exponential loss